In [129]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,scale
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import time
import pickle


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def stretch(x):
    return (x-x.min())/(x.max()-x.min())

In [91]:
test = np.array([20,50,60,21,52,64,8,40,70]).reshape(3,3)
print(test)

[[20 50 60]
 [21 52 64]
 [ 8 40 70]]


In [100]:
res0 = (test[:,0]-test[:,0].min())/(test[:,0].max()-test[:,0].min()).reshape(-1,1)
res1 = (test[:,1]-test[:,1].min())/(test[:,1].max()-test[:,1].min()).reshape(-1,1)
res2 = (test[:,2]-test[:,2].min())/(test[:,2].max()-test[:,2].min()).reshape(-1,1)

[[0.92307692 1.         0.        ]]
[[0.83333333 1.         0.        ]]
[[0.  0.4 1. ]]


In [103]:
print(np.concatenate((res0,res1,res2,),axis=0).T)

[[0.92307692 0.83333333 0.        ]
 [1.         1.         0.4       ]
 [0.         0.         1.        ]]


In [121]:
df = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test = pd.read_csv("../data/test.csv").drop("id",axis=1)

labels = df["target"].values
print(type(labels))
print(labels.shape)

unique_labels = np.unique(labels)
nb_labels = len(unique_labels)

df_train = df.drop("target", axis=1)

X = df_train.values
X_test = df_test.values

print(X.shape)
print(X_test.shape)

y = []
for label in labels:
    y.append(int(label[-1:]))
y = np.asarray(y)


print("Construction des centroïdes")
centroids = []
for label in unique_labels:
    same_labels = df_train.loc[df['target'] == label]
    same_labels = same_labels.values
    centroids.append(same_labels.sum(axis = 0)/same_labels.shape[0])
centroids = np.asarray(centroids)


print("Construction de la représentation dcDistance")

X_dc = pairwise_distances(X,centroids)
X_test_dc = pairwise_distances(X_test, centroids, metric='minkowski')

print("Construction terminée \n")

y_train = y

<class 'numpy.ndarray'>
(61878,)
(61878, 93)
(144368, 93)
Construction des centroïdes
Construction de la représentation dcDistance
Construction terminée 



In [133]:
res = (X_test_dc[:,0]-X_test_dc[:,0].min())/(X_test_dc[:,0].max()-X_test_dc[:,0].min()).reshape(-1,1)

for i in range(1,nb_labels):
    res_i = (X_test_dc[:,i]-X_test_dc[:,i].min())/(X_test_dc[:,i].max()-X_test_dc[:,i].min()).reshape(-1,1)
    res = np.concatenate((res,res_i), axis = 0)
res = res.T

In [135]:
print(res.shape)

(144368, 9)


In [140]:
res_final = np.array(list(map(lambda x : (x-x.min())/(x.max()-x.min()), res)))
res_final = np.array(list(map(lambda x : x/x.sum(), res_final)))


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


0.9999999999999999


In [4]:
start = time.time()

k_neighbors = 2


print("Apprentissage données en représentation dcDistance")

X_train_dc = scale(normalize(X_dc))

knn = KNeighborsClassifier(n_neighbors = k_neighbors)
knn.fit(X_train_dc, y_train)


print('done')

Apprentissage données en représentation dcDistance
done


In [ ]:
pred_proba = knn.predict_proba(X_test_dc)

print(pred_proba[:10])

pickle.dump(pred_proba, open('../data/dc.csv', 'wb'))

